In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import logging

from dataset.countdown_dataloader import Countdown
from dataset.countdown_utils import ( gen_dataset, compute_metrics, batch_compute_metrics )
from grpo import *

/Users/cormaccureton/mambaforge/envs/grpo-proj/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [4]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
# Create and save a tiny dataset with 5 samples
dataset_json_path = "simpler_countdown_data.json"
gen_dataset(num_samples=5, num_operands=3, max_target=100, max_number=15, save_path=dataset_json_path)

# Load the dataset
dataset = Countdown(dataset_json_path)

In [6]:
model_name = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Initialize the model with empty weights if needed
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [7]:
# Batch out dataset
batch_size = 3
batch_raw = dataset.get_batch(batch_size)

# Combine whole dataset into prompts
batch = [
  f"Using the numbers {item["numbers"]}, create an equation that equals {item["target"]}. Box your answer." 
  for item in batch_raw
  ]

In [8]:
# Use grpo sample outputs function
outputs_ids, outputs = sample_outputs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    G=3
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [9]:
# Calculate rewards for outputs
rewards, accuracies = batch_compute_metrics(
    outputs,
    queries=batch_raw
)

2025-04-07 12:56:53,370 - INFO - Rewards tensor shape: torch.Size([3, 3])
2025-04-07 12:56:53,370 - INFO - Accuracies tensor shape: torch.Size([3, 3])


In [10]:
# Print the outputs and rewards
for i, output in enumerate(outputs):
    print(f"Input: {batch[i]}")
    print(f"Output: {output}")
    print(f"Reward: {rewards[i]}")
    print(f"Accuracy: {accuracies[i]}")
    print("-" * 20)

Input: Using the numbers [1, 2, 4], create an equation that equals 5. Box your answer.
Output: ["Explain how a negative result was avoided and explain your reasoning. Let's create an equation using only the numbers [1, 2, 4] and try to solve it manually. We'll first attempt to use each number individually and see if we can reach 5.\n\n1. **First Try:**\n  ", "If you're unsure which numbers go where, look at 1, 2, and 4 as the answer.\nTo create an equation that equals 5 using the numbers 1, 2, and 4 as the answer, you can form one of the six permutations of these three digits as follows:\n\n1", 'Human beings are only humans, but are not necessarily animals.\nHuman logic\nAn important conclusion would be human are animals but not all animals are humans\nHow old do i have to be for my mom to ask me about the sun ? (0-100) A man is a human and not always a male']
Reward: tensor([0., 0., 0.])
Accuracy: tensor([0., 0., 0.])
--------------------
Input: Using the numbers [14, 6, 10], create a

In [11]:
# Compute GRPO advantage
advantage = calculate_grpo_advantage(
    rewards
)

In [12]:
print(f"Advantage: {advantage}")

Advantage: tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.5774, -0.5774,  1.1547]])


In [13]:
# Compute log probabilities
log_probs = compute_log_probs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    generated_ids=outputs_ids
)

In [14]:
# Calculate GRPO objective
grpo_objective = calculate_grpo_objective(
    model_log_probs=log_probs,
    old_model_log_probs=log_probs,  # Assuming old model is the same for this example
    ref_model_log_probs=log_probs,  # Assuming ref model is the same for this example
    advantages=advantage,
    eps=0.1,  # Epsilon for clipping
    beta=0.05,  # Beta for the objective function
)

print(f"GRPO Objective: {grpo_objective}")

GRPO Objective: tensor([ 0.0000e+00,  0.0000e+00, -1.9868e-08], grad_fn=<MeanBackward1>)


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [ ]:
updated_policy = grpo_iteration(
    query_batch_prompts=batch,
    query_batch_raw=batch_raw,
    policy_model=model,
    reference_model=model,  # Assuming reference model is the same for this example
    reward_model=batch_compute_metrics,
    tokenizer=tokenizer,
    optimizer=optimizer,
    G=3,
    eps=0.1,  # Epsilon for clipping
    beta=0.05,  # Beta for the objective function
    mu=3
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2025-04-07 12:57:07,871 - INFO - Rewards tensor shape: torch.Size([3, 3])
2025-04-07 12:57:07,875 - INFO - Accuracies tensor shape: torch.Size([3, 3])
2025-04-07 12:57:21,714 - INFO - Update iteration: 1/3
2025-04-07 12:57:29,404 - INFO - Loss: 3.311369312086754e-08
